In [8]:
from graphdriver.commons import setup
import hpbandster.core.result as hpres
from graphdriver.utils import paths
from scipy.stats import pearsonr
import numpy as np
import pandas as pd


In [9]:
def get_scores(res_budget):
    val_score = []
    test_score = []
    for r in res_budget:
        val = r['info']['pr_auc_mean_val']
        test = r['info']['pr_auc_mean_test']
        if val == 0 or test == 0:
            continue
        val_score.append(val)
        test_score.append(test)
    corr, _ = pearsonr(val_score, test_score)
    return corr

In [10]:
corrs_cancers = []
nts = setup.network_types()[:3]
nts.reverse()
# nts = [['genes']]
for c in setup.cancer_types():
    corrs_nt = []
    for nt  in nts:
        corrs_outer = []
        for outer_fold in range(10):
            path = paths.results_hpo(cancer=c, network_type=nt, outer_fold=outer_fold)
            res = hpres.logged_results_to_HBS_result(path)
            res_budget = res.get_all_runs()
            res_budget = [r for r in res_budget if r.budget == 4]
            corr = get_scores(res_budget)
            corrs_outer.append(corr)
        corrs_nt.append(np.mean(corrs_outer))
    corrs_cancers.append(corrs_nt)

In [11]:
df = pd.DataFrame(corrs_cancers, columns=['MGNNdriver', 'a)', 'b)'])
df.index = ["\textbf{" + c.upper() + "}" for c in setup.cancer_types()]
df.loc['\textbf{MEAN}'] = df.mean()
df = df.round(3)

df.to_latex("hpo_correlation_results.tex", escape=False)


/tmp/ipykernel_23102/1290622210.py:6: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.to_latex("hpo_correlation_results.tex", escape=False)
